In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile9 = pd.read_pickle("DDD_Train_Decile9.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile9.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile9, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3231 - acc: 0.8693 - val_loss: 0.2680 - val_acc: 0.9083
Epoch 39/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3208 - acc: 0.8710 - val_loss: 0.2641 - val_acc: 0.9065
Epoch 40/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3244 - acc: 0.8679 - val_loss: 0.2677 - val_acc: 0.9118
Epoch 41/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3184 - acc: 0.8684 - val_loss: 0.2707 - val_acc: 0.9136
Epoch 42/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3196 - acc: 0.8666 - val_loss: 0.2627 - val_acc: 0.9083
Epoch 43/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3195 - acc: 0.8777 - val_loss: 0.2598 - val_acc: 0.9101
Epoch 44/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3121 - acc: 0.8768 - val_loss: 0.2567 - val_acc: 0.9189
Epoch 45/100
2264/2264 [==============================] - 5s 

Epoch 98/100
2264/2264 [==============================] - 6s 2ms/step - loss: 0.3005 - acc: 0.8781 - val_loss: 0.2421 - val_acc: 0.9083
Epoch 99/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3069 - acc: 0.8772 - val_loss: 0.2425 - val_acc: 0.9065
Epoch 100/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2969 - acc: 0.8768 - val_loss: 0.2418 - val_acc: 0.9101
Test accuracy: 0.9100529093170503
Train on 2264 samples, validate on 567 samples
Epoch 1/150
2264/2264 [==============================] - 6s 2ms/step - loss: 0.6274 - acc: 0.5967 - val_loss: 0.5450 - val_acc: 0.6402
Epoch 2/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4862 - acc: 0.7836 - val_loss: 0.3893 - val_acc: 0.8519
Epoch 3/150
2264/2264 [==============================] - 6s 2ms/step - loss: 0.4234 - acc: 0.8454 - val_loss: 0.3288 - val_acc: 0.8818
Epoch 4/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3634 - acc: 0.8617 - val_l

2264/2264 [==============================] - 6s 3ms/step - loss: 0.2846 - acc: 0.8821 - val_loss: 0.2232 - val_acc: 0.9136
Epoch 57/150
2264/2264 [==============================] - 6s 3ms/step - loss: 0.2819 - acc: 0.8816 - val_loss: 0.2240 - val_acc: 0.9171
Epoch 58/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2805 - acc: 0.8825 - val_loss: 0.2237 - val_acc: 0.9153
Epoch 59/150
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2868 - acc: 0.8777 - val_loss: 0.2234 - val_acc: 0.9171
Epoch 60/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2840 - acc: 0.8790 - val_loss: 0.2233 - val_acc: 0.9171
Epoch 61/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2837 - acc: 0.8772 - val_loss: 0.2230 - val_acc: 0.9136
Epoch 62/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2837 - acc: 0.8807 - val_loss: 0.2232 - val_acc: 0.9153
Epoch 63/150
2264/2264 [==============================] - 5s 

Epoch 25/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2967 - acc: 0.8785 - val_loss: 0.2316 - val_acc: 0.9171
Epoch 26/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2972 - acc: 0.8785 - val_loss: 0.2297 - val_acc: 0.9206
Epoch 27/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2892 - acc: 0.8821 - val_loss: 0.2296 - val_acc: 0.9171
Epoch 28/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2913 - acc: 0.8754 - val_loss: 0.2297 - val_acc: 0.9224
Epoch 29/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2951 - acc: 0.8772 - val_loss: 0.2293 - val_acc: 0.9206
Epoch 30/150
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3007 - acc: 0.8785 - val_loss: 0.2291 - val_acc: 0.9224
Epoch 31/150
2264/2264 [==============================] - 6s 3ms/step - loss: 0.2982 - acc: 0.8803 - val_loss: 0.2297 - val_acc: 0.9153

Epoch 00031: ReduceLROnPlateau reducing learnin

...

Epoch 86/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2805 - acc: 0.8825 - val_loss: 0.2222 - val_acc: 0.9189
Epoch 87/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2778 - acc: 0.8807 - val_loss: 0.2223 - val_acc: 0.9189
Epoch 88/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2842 - acc: 0.8768 - val_loss: 0.2221 - val_acc: 0.9189
Epoch 89/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2831 - acc: 0.8878 - val_loss: 0.2222 - val_acc: 0.9189
Epoch 90/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2795 - acc: 0.8794 - val_loss: 0.2221 - val_acc: 0.9189
Epoch 91/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2846 - acc: 0.8790 - val_loss: 0.2220 - val_acc: 0.9189
Epoch 92/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2853 - acc: 0.8834 - val_loss: 0.2219 - val_acc: 0.9206
Epoch 93/200
2264/2264 [========================

Epoch 146/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2716 - acc: 0.8777 - val_loss: 0.2214 - val_acc: 0.9136
Epoch 147/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2804 - acc: 0.8821 - val_loss: 0.2217 - val_acc: 0.9136
Epoch 148/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2785 - acc: 0.8777 - val_loss: 0.2216 - val_acc: 0.9136
Epoch 149/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2777 - acc: 0.8852 - val_loss: 0.2219 - val_acc: 0.9136
Epoch 150/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2822 - acc: 0.8807 - val_loss: 0.2217 - val_acc: 0.9136
Epoch 151/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.2854 - acc: 0.8847 - val_loss: 0.2220 - val_acc: 0.9136
Epoch 152/200
2264/2264 [==============================] - 6s 3ms/step - loss: 0.2712 - acc: 0.8843 - val_loss: 0.2221 - val_acc: 0.9136
Epoch 153/200
2264/2264 [================

2264/2264 [==============================] - 5s 2ms/step - loss: 0.3165 - acc: 0.8706 - val_loss: 0.2615 - val_acc: 0.9065
Epoch 66/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3181 - acc: 0.8688 - val_loss: 0.2590 - val_acc: 0.9083
Epoch 67/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3217 - acc: 0.8719 - val_loss: 0.2572 - val_acc: 0.9065
Epoch 68/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3249 - acc: 0.8693 - val_loss: 0.2614 - val_acc: 0.9101
Epoch 69/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3211 - acc: 0.8679 - val_loss: 0.2597 - val_acc: 0.9136
Epoch 70/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3190 - acc: 0.8746 - val_loss: 0.2587 - val_acc: 0.9101
Epoch 71/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.3167 - acc: 0.8728 - val_loss: 0.2580 - val_acc: 0.9118
Epoch 72/100
2264/2264 [==============================] - 5s 

2264/2264 [==============================] - 6s 2ms/step - loss: 0.2778 - acc: 0.8812 - val_loss: 0.2182 - val_acc: 0.9206
Epoch 83/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2742 - acc: 0.8785 - val_loss: 0.2181 - val_acc: 0.9206
Epoch 84/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2811 - acc: 0.8830 - val_loss: 0.2180 - val_acc: 0.9206
Epoch 85/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2859 - acc: 0.8790 - val_loss: 0.2182 - val_acc: 0.9206
Epoch 86/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2720 - acc: 0.8852 - val_loss: 0.2184 - val_acc: 0.9206
Epoch 87/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2732 - acc: 0.8812 - val_loss: 0.2183 - val_acc: 0.9206
Epoch 88/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2787 - acc: 0.8763 - val_loss: 0.2180 - val_acc: 0.9206
Epoch 89/200
2264/2264 [==============================] - 6s 

2264/2264 [==============================] - 6s 2ms/step - loss: 0.2673 - acc: 0.8794 - val_loss: 0.2180 - val_acc: 0.9224
Epoch 143/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2780 - acc: 0.8852 - val_loss: 0.2181 - val_acc: 0.9206
Epoch 144/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.2795 - acc: 0.8772 - val_loss: 0.2183 - val_acc: 0.9242
Epoch 145/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2803 - acc: 0.8803 - val_loss: 0.2184 - val_acc: 0.9242
Epoch 146/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2766 - acc: 0.8816 - val_loss: 0.2182 - val_acc: 0.9224
Epoch 147/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2761 - acc: 0.8821 - val_loss: 0.2182 - val_acc: 0.9224
Epoch 148/200
2264/2264 [==============================] - 6s 2ms/step - loss: 0.2759 - acc: 0.8763 - val_loss: 0.2184 - val_acc: 0.9242
Epoch 149/200
2264/2264 [==============================

In [ ]:
0.86434

In [ ]:
0.86